# prepare_dataset

Track Zeno's face images using an [exisiting tool](https://github.com/IntelligentBehaviourUnderstandingGroup/dlib_and_chehra_stuff) and prepare them for manual annotation.

## Step 1: Import modules

In [1]:
import os
import sys
import cv2
import glob
import time
try:
    from ConfigParser import ConfigParser    # If using Python 2.7
except ImportError:
    from configparser import ConfigParser    # If using Python 3.5
config = ConfigParser()
config.read('prepare_dataset.ini')
sys.path.append(os.path.realpath(config.get('facial_landmark_tracker', 'repository_path')))
import ibug_face_tracker
from zeno_face_tracker_helpers import *
print('All modules imported.')

All modules imported.


## Step 2: Initialise the landmark localiser

In [2]:
tracker = ibug_face_tracker.FaceTracker(os.path.realpath(config.get('facial_landmark_tracker', 
                                                                    'ert_model_path')), 
                                        os.path.realpath(config.get('facial_landmark_tracker', 
                                                                    'auxiliary_model_path')))
tracker.face_detection_scale = config.getfloat('facial_landmark_tracker', 'face_detection_scale')
tracker.minimum_face_size = config.getint('facial_landmark_tracker', 'minimum_face_size')
tracker.hard_failure_threshold = -1e6
tracker.estimate_head_pose = False
tracker.eye_iterations = 0
print('Facial landmark localiser initialised.')

Facial landmark localiser initialised.


## Step 3: Localise Facial Landmarks

In [3]:
# Enumerate jobs
source_images = sorted(glob.glob(os.path.realpath(os.path.join('./dataset', 'cam*', '*.png'))))
source_images = [x for x in source_images if '.pts.png' not in x]
print('%d images to be processed.' % len(source_images))

# Get landmarks
successes = 0
last_check_time = time.time()
for idx, image_path in enumerate(source_images):
    image = cv2.imread(image_path)
    tracker.reset()
    tracker.track(image)
    if tracker.has_facial_landmarks:
        pts_path = os.path.splitext(image_path)[0] + '.init.pts'
        save_pts(pts_path, tracker.facial_landmarks)
        tracker.plot_current_result(image)
        rendering_path = pts_path + '.png'
        cv2.imwrite(rendering_path, image)
        successes += 1
    current_time = time.time()
    if current_time - last_check_time > 10.0:
        last_check_time = current_time
        print('%d images have been processed.' % (idx + 1))
print('Done, all %d images have been processed, %d of which have landmarks localised.' % 
      (len(source_images), successes))

1200 images to be processed.
112 images have been processed.
219 images have been processed.
334 images have been processed.
450 images have been processed.
566 images have been processed.
683 images have been processed.
796 images have been processed.
921 images have been processed.
1027 images have been processed.
1141 images have been processed.
Done, all 1200 images have been processed, 1048 of which have landmarks localised.
